In [49]:
import heapq # Priority queue
from collections import deque

# **Uninformed search algorithms**

In [50]:
# Maze
maze = [
    [0, 0, 1, 1, 0],
    [1, 0, 0, 1, 0],
    [1, 1, 0, 1, 1],
    [0, 0, 0, 0, 0],
    [1, 1, 1, 0, 1],
    [1, 1, 1, 'T', 1]
]
start = (0,0)

## **Breadth-first search algorithm**

In [51]:
def BFS(maze, start):
  rows = len(maze)  # Number of rows of the maze
  columns = len(maze[0])  # Number of columns of the maze
  print(f"Rows: {rows} Columns: {columns}")

  # Possible movements: up, down, left, right
  movements_x = [-1, 1, 0, 0]
  movements_y = [0, 0, 1, -1]

  # Start position
  start_x, start_y = start

  # Create BFS queues
  row_queue = deque()
  column_queue = deque()

  # Variables to track movements
  moves_count = 0
  nodes_left_in_layer = 1 # Number of nodes for exploring in the current layer
  nodes_in_next_layer = 0 # Number of nodes in the next layer
  end = False  # Flag for reaching 'T'

  # Visited matrix
  visited = [[False for _ in range(columns)] for _ in range(rows)]

  # Start BFS
  row_queue.append(start_x) # Add the starting position to the queue
  column_queue.append(start_y)
  visited[start_x][start_y] = True

  while len(row_queue) > 0:
    # Removes the value from the queue
    x = row_queue.popleft()
    y = column_queue.popleft()

    if maze[x][y] == 'T':
      return f"Minimum number of moves: {moves_count}"

    # Update nodes_in_next_layer properly
    nodes_in_next_layer = explore_neighbors(x, y, movements_x, movements_y, visited,
                        row_queue, column_queue, nodes_in_next_layer, maze, rows, columns)
    nodes_left_in_layer -= 1

    if nodes_left_in_layer == 0:

      nodes_left_in_layer = nodes_in_next_layer
      nodes_in_next_layer = 0
      moves_count += 1

  return "No path found"  # No path found

# Explore negihbors and returns
def explore_neighbors(x, y, movements_x, movements_y, visited, row_queue,
                    column_queue, nodes_in_next_layer, maze, rows, columns):
  for i in range(4):
    next_x = x + movements_x[i]
    next_y = y + movements_y[i]

    # Check bounds of the maze
    if next_x < 0 or next_y < 0:
      continue
    if next_x >= rows or next_y >= columns:
      continue

    # Avoid visited locations or blocked cells (1)
    if visited[next_x][next_y]:
      continue
    if maze[next_x][next_y] == 1:
      continue

    # Add the coordinates to the queues
    row_queue.append(next_x)
    column_queue.append(next_y)
    visited[next_x][next_y] = True # Mark the position as visited
    nodes_in_next_layer += 1  # Update count

  return nodes_in_next_layer  # Return updated count

In [52]:
BFS(maze, start)

Rows: 6 Columns: 5


'Minimum number of moves: 8'

## **Depth-first search algorithm**

In [53]:
def DFS(maze, start):
  rows = len(maze)  # Number of rows in the maze
  columns = len(maze[0])  # Number of columns in the maze
  print(f"Rows: {rows} Columns: {columns}")

  # Possible movements: up, down, left, right
  movements_x = [-1, 1, 0, 0]
  movements_y = [0, 0, 1, -1]

  # Start position
  start_x, start_y = start

  # Create DFS stack (two stacks)
  row_stack = deque()
  column_stack = deque()

  # Craete a visited matrix (filling with False values)
  visited = [[False for _ in range(columns)] for _ in range(rows)]

  # Start DFS
  row_stack.append(start_x)  # Add the starting first position to the stack
  column_stack.append(start_y) # Add the other position to the stack
  visited[start_x][start_y] = True # Mark as visited

  # Path tracking (not counting layers like BFS)
  moves_count = 0

  while len(row_stack) > 0:
      # Removes the value from the stack (LIFO order)
      x = row_stack.pop()
      y = column_stack.pop()

      if maze[x][y] == 'T':
          return f"The goal was found in {moves_count} moves."

      # Explore neighbors
      for i in range(4):
          new_x = x + movements_x[i]
          new_y = y + movements_y[i]

          if check_valid_pos(new_x, new_y, maze, visited, rows, columns):
              row_stack.append(new_x)
              column_stack.append(new_y)
              visited[new_x][new_y] = True

      moves_count += 1  # Increment moves each time a new node is visited

  return -1  # No path found


# Function that checks if the position is valid
def check_valid_pos(x, y, maze, visited, rows, columns):

  # Check that the coordinates are valid
  if x < 0 or y < 0 or x >= rows or y >= columns:
      return False
  #  Check if the position is already visited or is a wall
  if visited[x][y] or maze[x][y] == 1: # It checks if there is a wall (1)
      return False
  return True

In [54]:
DFS(maze, start)

Rows: 6 Columns: 5


'The goal was found in 11 moves.'

## **Uniform cost search algorithm**

In [55]:
# Maze with cost in a dictionary form
maze_with_cost = {
  'A': [('B', 1), ('C', 4)],
  'B': [('D', 5), ('E', 2)],
  'C': [('F', 3), ('G', 4)],
  'D': [('H', 3)],
  'E': [('H', 6)],
  'F': [('I', 4)],
  'G': [('J', 2)],
  'H': [('I', 1)],
  'I': [('J', 2)]
}

In [56]:
def UCS(graph, start, end):
  # Priority queue stores tuples of (cost, node, path)
  priority_queue = [(0, start, [])]
  visited = set()  # Set to keep track of visited nodes

  while priority_queue:
      # Extract the node with the lowest cost from the priority queue
      cost, node, path = heapq.heappop(priority_queue)

      if node in visited: # If the node has already been visited, we skip it
          continue
      visited.add(node)  # Mark the node as visited

      # Update the path with the current node
      path = path + [node]

      # If we reached the end node, then return the result
      if node == end:
          # Prints a message
          return f"Shortest path: {path}. The path would cost: {cost}"

      # Retrieves all neighbors of node
      for neighbor, edge_cost in graph.get(node, []):
          if neighbor not in visited: # If the neighbor has not been visited, we add it to the priority queue

              # Push the neighbor into the priority queue with updated cost and path
              heapq.heappush(priority_queue, (cost + edge_cost, neighbor, path))

  return "No path found"

In [57]:
UCS(maze_with_cost, 'A', 'J')

"Shortest path: ['A', 'C', 'G', 'J']. The path would cost: 10"

# **Informed search algorithms**

In [58]:
# Heruistics
heuristic_values = {
    'A': 7,
    'B': 6,
    'C': 3,
    'D': 5,
    'E': 4,
    'F': 2,
    'G': 1,
    'H': 3,
    'I': 1,
    'J': 0
}

## **Greedy algorithm**

In [59]:
def greedy_search(graph, start, goal, heuristic_values):

    # Set to keep track of visited nodes
    visited = set()

    # Priority queue storing tuples: (heuristic, current_node, path, cost)
    priority_queue = [(heuristic_values[start], start, [start], 0)]

    while priority_queue:
        # Pop the node with the lowest heuristic value
        current_h, current_node, path, cost = heapq.heappop(priority_queue)

        if current_node in visited:
            continue
        visited.add(current_node)

        # Check if we've reached the goal
        if current_node == goal:
            return f"Path found: {path}. Total cost: {cost}"

        # Expand neighbors
        for neighbor, edge_cost in graph.get(current_node, []):
            if neighbor not in visited:
                new_cost = cost + edge_cost # Computes the new cost
                # Get the heuristic value for the neighbor (if not found put a high value)
                new_h = heuristic_values.get(neighbor, float('inf'))
                heapq.heappush(priority_queue, (new_h, neighbor, path + [neighbor], new_cost))

    return "No path found"

In [60]:
greedy_search(maze_with_cost, 'A', 'J', heuristic_values)

"Path found: ['A', 'C', 'G', 'J']. Total cost: 10"

## **A* algorithm**

In [61]:
def a_star_search(graph, start, goal, heuristic_values):

  open_list = [] # Create a list that stores tuples
  initial_heuristic = heuristic_values.get(start, float('inf'))
  heapq.heappush(open_list, (initial_heuristic, 0, start, [start]))

  # The closed_set keeps track of nodes that have been completely visited
  closed_set = set()

  while open_list:
      # Pop the node with the smallest total estimated cost from the priority queue
      total_estimated_cost, path_cost, current_node, path_so_far = heapq.heappop(open_list)

      # Check if we have reached the goal node
      if current_node == goal:
          return f"Path found: {path_so_far}. Total cost: {path_cost}"

      # If the node was already visited we skip it
      if current_node in closed_set:
          continue
      closed_set.add(current_node)

      # Expand the current node's neighbors
      for neighbor, edge_cost in graph.get(current_node, []):
          # Skip neighbor if it has already been processed
          if neighbor in closed_set:
              continue

          new_path_cost = path_cost + edge_cost # Compute new cost from start to neighbor
          heuristic_neighbor = heuristic_values.get(neighbor, float('inf')) # Retrieve the heuristic value for the neighbor

          # Compute the total estimated cost for the neighbor
          new_total_estimated_cost = new_path_cost + heuristic_neighbor

          new_path = path_so_far + [neighbor] # Construct the new path

          # Push the neighbor into the priority queue
          heapq.heappush(open_list, (new_total_estimated_cost, new_path_cost, neighbor, new_path))

  return "No path found"

In [62]:
a_star_search(maze_with_cost, 'A', 'J', heuristic_values)

"Path found: ['A', 'C', 'G', 'J']. Total cost: 10"

## **A* tree algorithm**

In [63]:
def a_star_search(graph, start, goal, heuristic_values):
  # Start with the starting node
  initial_h = heuristic_values.get(start, float('inf'))  # Use the heuristic value for the start node
  open_list = [(initial_h, 0, start, [start])]

  # Create set
  closed_set = set()

  while open_list:
      # Pop the node with the smallest value
      total_cost, cost_so_far, current, path = heapq.heappop(open_list)

      # If the current node is the goal, return the path and total cost
      if current == goal:
          return f"Path found: {path}. Total cost: {cost_so_far}" # Print path


      # If the node has already been visited the skip it.
      if current in closed_set:
          continue
      closed_set.add(current)

      # Expand neighbors of the current node
      for neighbor, edge_cost in graph.get(current, []):
          if neighbor in closed_set:
              continue  # Skip already processed neighbors

          new_cost = cost_so_far + edge_cost  # Update cumulative cost g

          # Get the heuristic value for the neighbor (use infinity if missing)
          new_h = heuristic_values.get(neighbor, float('inf'))
          new_total_cost = new_cost + new_h # formula: f = g + h
          new_path = path + [neighbor] # Extend the current path

          # Push the neighbor into the priority queue
          heapq.heappush(open_list, (new_total_cost, new_cost, neighbor, new_path))

  return "No path found"

In [64]:
a_star_search(maze_with_cost, 'A', 'J',  heuristic_values)

"Path found: ['A', 'C', 'G', 'J']. Total cost: 10"